In [1]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.2 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.2.1
    Uninstalling networkx-3.2.1:
      Successfully uninstalled networkx-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [2]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random         #Para generar valores aleatorios


#http://elib.zib.de/pub/mp-testdata/tsp/tsplib/
#Documentacion :
  # http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp95.pdf
  # https://tsplib95.readthedocs.io/en/stable/pages/usage.html
  # https://tsplib95.readthedocs.io/en/v0.6.1/modules.html
  # https://pypi.org/project/tsplib95/

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

#Coordendas 51-city problem (Christofides/Eilon)
#file = "eil51.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/eil51.tsp.gz", file)

#Coordenadas - 48 capitals of the US (Padberg/Rinaldi)
#file = "att48.tsp" ; urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/att48.tsp.gz", file)

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [3]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [4]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)

15

In [63]:

#Funcionas basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

sol_temporal = crear_solucion(Nodos)
solucion = crear_solucion(Nodos)
print(solucion)
distancia_total(sol_temporal, problem), sol_temporal

[0, 30, 1, 18, 36, 13, 38, 8, 24, 41, 16, 6, 17, 34, 9, 27, 39, 26, 2, 33, 20, 29, 3, 10, 19, 14, 11, 32, 37, 23, 4, 5, 25, 7, 12, 40, 31, 22, 28, 21, 15, 35]


(5088,
 [0,
  9,
  17,
  38,
  37,
  19,
  31,
  29,
  27,
  16,
  10,
  22,
  21,
  3,
  36,
  2,
  6,
  8,
  35,
  12,
  41,
  26,
  40,
  5,
  20,
  23,
  28,
  13,
  4,
  15,
  32,
  30,
  33,
  25,
  11,
  24,
  18,
  39,
  1,
  14,
  7,
  34])

In [72]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 4900
Distancia Mejor Solucion Local: 4416


In [70]:
###############################################################################
# BUSQUEDA LOCAL MEJORADA 1
###############################################################################

#Intercambio aleatorio de dos ciudades de la ruta

def intercambiar_ciudades(solucion):

    # Elijo dos ciudades aleatorias
    i, j = random.sample(range(len(solucion)), 2)

    #Copio la solución propocionada a una variable nueva
    nueva_ruta = solucion[:]

    #Intercambio de la posición de la ciudad
    nueva_ruta[i], nueva_ruta[j] = nueva_ruta[j], nueva_ruta[i]

    return i, j, nueva_ruta

x, y, nueva_solucion = intercambiar_ciudades(solucion)


print(solucion)
print('')
print(f"Intercambio la ciudad de índice {x} con la de posición {y}")
print(nueva_solucion)
print('')
print("Distancia Mejor Solucion Local - Intercambio:", distancia_total(nueva_solucion, problem))

[0, 30, 1, 18, 36, 13, 38, 8, 24, 41, 16, 6, 17, 34, 9, 27, 39, 26, 2, 33, 20, 29, 3, 10, 19, 14, 11, 32, 37, 23, 4, 5, 25, 7, 12, 40, 31, 22, 28, 21, 15, 35]

Intercambio la ciudad de índice 4 con la de posición 21
[0, 30, 1, 18, 29, 13, 38, 8, 24, 41, 16, 6, 17, 34, 9, 27, 39, 26, 2, 33, 20, 36, 3, 10, 19, 14, 11, 32, 37, 23, 4, 5, 25, 7, 12, 40, 31, 22, 28, 21, 15, 35]

Distancia Mejor Solucion Local - Intercambio: 4846


In [71]:
###############################################################################
# BUSQUEDA LOCAL MEJORADA 2
###############################################################################

#Insertar una ciudad aleatoria en una posición diferente

def por_insercion(solucion):
    # Elijo dos ciudades aleatorias
    i, j = sorted(random.sample(range(len(solucion)), 2))

    #Inserto la ciudad de posicion i a la posicion j, dividiendo en 4 tramos la ruta
    nueva_ruta = solucion[:i] + solucion[i+1:j] + [solucion[i]] + solucion[j:]

    return i, j, nueva_ruta


x, y, nueva_solucion = por_insercion(solucion)

print(solucion)
print('')
print(f"Inserto la ciudad de índice {x} a la posición {y}")
print(nueva_solucion)
print('')
print("Distancia Mejor Solucion Local - Inserción:", distancia_total(nueva_solucion, problem))

[0, 30, 1, 18, 36, 13, 38, 8, 24, 41, 16, 6, 17, 34, 9, 27, 39, 26, 2, 33, 20, 29, 3, 10, 19, 14, 11, 32, 37, 23, 4, 5, 25, 7, 12, 40, 31, 22, 28, 21, 15, 35]

Inserto la ciudad de índice 1 a la posición 37
[0, 1, 18, 36, 13, 38, 8, 24, 41, 16, 6, 17, 34, 9, 27, 39, 26, 2, 33, 20, 29, 3, 10, 19, 14, 11, 32, 37, 23, 4, 5, 25, 7, 12, 40, 31, 30, 22, 28, 21, 15, 35]

Distancia Mejor Solucion Local - Inserción: 4780
